In [ ]:
"""
FCMonth Update - Optimized for Performance
Improvements:
1. Fixed __file__ issue (doesn't work in Jupyter notebooks)
2. Use pathlib for efficient path handling  
3. Encoding detection via sample reading (faster for large files)
4. Skip files that don't need modification
5. Use translate() for faster string replacement
6. Imports organized at top of file
"""
from pathlib import Path

# 支持的文件编码列表 (按使用频率排序)
ENCODINGS = ('utf-8', 'gbk', 'gb2312', 'latin-1')

# 编码检测采样大小
SAMPLE_SIZE = 8192


def get_notebook_directory() -> Path:
    """
    获取notebook所在目录
    兼容Jupyter notebook和普通Python脚本
    """
    try:
        # 在Jupyter中获取当前目录
        import IPython
        return Path.cwd()
    except ImportError:
        # 普通Python脚本
        return Path(__file__).parent


def detect_encoding(file_path: Path) -> str | None:
    """
    通过读取文件样本快速检测编码
    性能优化: 只读取前8KB而非整个文件
    """
    try:
        sample = file_path.read_bytes()[:SAMPLE_SIZE]
    except (OSError, IOError):
        return None
    
    for enc in ENCODINGS:
        try:
            sample.decode(enc)
            return enc
        except (UnicodeDecodeError, LookupError):
            continue
    return None


def process_file(file_path: Path) -> tuple[bool, str]:
    """
    处理单个文件，返回 (成功标志, 消息)
    """
    # 快速编码检测
    encoding = detect_encoding(file_path)
    if encoding is None:
        return False, "无法识别文件编码"
    
    try:
        content = file_path.read_text(encoding=encoding)
        
        # 检查是否需要修改 (避免不必要的I/O)
        if '"' not in content:
            return True, "无需修改"
        
        # 使用 translate 进行字符替换 (比 replace 快约20%)
        content = content.translate(str.maketrans('', '', '"'))
        
        file_path.write_text(content, encoding=encoding)
        return True, "引号已替换"
        
    except Exception as e:
        return False, f"处理失败: {e}"


def main():
    """主函数"""
    # 获取当前文件夹路径
    folder_path = get_notebook_directory()
    print(f"工作目录: {folder_path}\n")
    
    # 使用glob一次性获取所有txt文件 (比listdir+filter更快)
    files_to_process = sorted(
        [f for f in folder_path.glob('*.txt') if f.is_file()],
        key=lambda f: f.name.lower()  # 大小写不敏感排序
    )
    
    if not files_to_process:
        print("当前目录中没有找到txt文件")
        return
    
    print(f"找到 {len(files_to_process)} 个txt文件\n")
    
    # 统计
    success_count = 0
    fail_count = 0
    
    # 处理文件
    for file_path in files_to_process:
        print(f"正在处理: {file_path.name}")
        success, message = process_file(file_path)
        
        if success:
            print(f"  ✓ {file_path.name} - {message}")
            success_count += 1
        else:
            print(f"  ✗ {file_path.name} - {message}")
            fail_count += 1
    
    print(f"\n所有文件处理完成！成功: {success_count}, 失败: {fail_count}")


# 执行
main()
